# The Penalty Method

Click on the link for the interactive version:
[![Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/pjohno/MATH60082/master?labpath=notebooks/MATH60082%20-%20Labs%20Week%2010%20-%20Part%20II.ipynb)

Here we aim to solve the classic American put option problem using the so called penalty method. 
We must solve the problem
$$
\frac{\partial P}{\partial t} + \frac12\sigma^2S^2\frac{\partial^2 P}{\partial S^2} + rS\frac{\partial P}{\partial S} - rP = 0
$$
with
$$
P(S,T) = \max(X-S,0),
$$
$$
P(S,t) \geq X-S,
$$
and
$$
S_0 = 9.735 \text{ , } X=10  \text{ , } T=1 \text{ , } r=0.05 \text{ , } \sigma=0.4 ~.
$$
The penalty method works by solving a slightly different problem, which can be shown to converge to the true solution. The alternative problem we solve is given by 
$$
\frac{\partial P}{\partial t} + \frac12\sigma^2S^2\frac{\partial^2 P}{\partial S^2} + rS\frac{\partial P}{\partial S} - rP + \rho\max((X - S) - P , 0 ) = 0
$$
where in the limit as $\rho \rightarrow \infty$, the value function $P$ satisfies
$$
P(S,t) \geq X-S.
$$
The algorithm works as follows
1. Write down a standard finite difference scheme for the original PDE (including the boundaries)
2. Take a guess at the solution $v_j^q$ at the current time step, where $v_j^q$ is the $q$th guess.
3. If $v_j^q < X - S_j$ adjust the original finite difference approximation to
$$
\hat b_j = b_j - \rho
$$
$$
\hat d_j = d_j - \rho ( X - S_j )
$$ 
4. Solve new system with a Thomas algorithm to find $v_j^{q+1}$.
5. Check for convergence in $||v^q - v^{q+1}||$.

In order to guarantee that our solution is accurate to the level `tol`, we can choose 
$$
\rho = \frac{1}{tol}. 
$$

A full implementation of this method is also available on the web [click here](https://github.com/pjohno/MATH60082/blob/master/main/math60082_example_sheet_8_penalty.cpp).

First include standard libraries:

In [1]:
#include "math60082_lab_plot_loader.hpp"
#include <iostream>
#include <vector>
#include <chrono>
#include <string>
#include <fstream>
#include <cmath>
#include <algorithm>
using namespace std;

and an implementation of the Thomas solver.

In [2]:
/* 
 *    ON INPUT:
 *    a, b and c -- are the diagonals of the matrix
 *    rhs        -- is the right hand side
 *    ON OUTPUT:
 *    a, b, c             -- unchanged
 *    rhs                 -- solution to Ax=b
 */
void thomasSolve(const std::vector<double> &a,const std::vector<double> &b_,const std::vector<double> &c,std::vector<double> &rhs)
{
    int n=a.size();
    std::vector<double> b(n);
    // initial first value of b
    b[0]=b_[0];
    for(int j=1;j<n;j++)
    {
        b[j]=b_[j]-c[j-1]*a[j]/b[j-1];  
        rhs[j]=rhs[j]-rhs[j-1]*a[j]/b[j-1];
    }
    // calculate solution
    rhs[n-1]=rhs[n-1]/b[n-1];
    for(int j=n-2;j>=0;j--)
        rhs[j]=(rhs[j]-c[j]*rhs[j+1])/b[j];
}


Next we have packaged the American put option solver up into a function. This code has been adapted from the European put option solver with SOR, so the inputs/outputs are the same except the over relaxation parameter has been swapped for the penalty. There is full flexibility on the choice of numerical parameters.

In [3]:
/** 
@brief Return the value of an American Put option using the Crank-Nicolson method with Thomas solver
ON INPUT:
@param S0          -- initial stock price
@param X           -- exercise (strike) price
@param T           -- Time to expiry (years)
@param r           -- interest rate (per annum)
@param sigma       -- volatility (per annum^12)
@param iMax        -- number of time steps
@param jMax        -- number of space steps
@param SMax        -- Maximum value of S
@param penalty     -- is the penalty parameter, should be around 1/tol
@param tol         -- is the tolerance level, should be || V || / 10^l where l is the required digits of accuracy 
@param iterMax     -- is maximum iterations
ON OUTPUT:
@return the value of a American put option at \f$ S=S0 \f$, \f$ t=0 \f$
**/
double americanPut_CN(double S0,double X,double T,double r,double sigma,
                      int iMax,int jMax,double SMax,double penalty,double tol,int iterMax)
{
    double dS=SMax/jMax;
    double dt=T/iMax;
    // create storage for the stock price and option price (old and new)
    std::vector<double> S(jMax+1),vOld(jMax+1),vNew(jMax+1);
    // setup and initialise the stock price 
    for(int j=0;j<=jMax;j++)
    {
        S[j] = j*dS;
    }
    // reset initial condition
    for(int j=0;j<=jMax;j++)
    {
        vOld[j] = std::max(X-S[j],0.);
        vNew[j] = std::max(X-S[j],0.);
    }
    // run through timesteps
    for(int i=iMax-1;i>=0;i--)
    {
        // declare vectors for matrix equations
        std::vector<double> a(jMax+1),b(jMax+1),c(jMax+1),d(jMax+1);
        // set up matrix equations a[j]=
        a[0] = 0.;b[0] = -1./dt - 0.5*r;c[0] = 0.;
        d[0] = (-1./dt + 0.5*r)*vOld[0];
        for(int j=1;j<jMax;j++)
        {
            a[j] = 0.25*(sigma*sigma*j*j-r*j);
            b[j] =-0.5*sigma*sigma*j*j - 0.5*r - 1./dt;
            c[j] = 0.25*(sigma*sigma*j*j+r*j);
            d[j] =-a[j]*vOld[j-1] - (b[j]+2./dt)*vOld[j] - c[j]*vOld[j+1];
        }
        a[jMax] = 0.;b[jMax] = 1.;c[jMax] = 0.;
        d[jMax] = 0.;
        // solve with penalty method
        int iter;
        for(iter=0;iter<iterMax;iter++)
        {
            // create new vectors containing the FD approximations
            std::vector<double> bHat(jMax+1),dHat(jMax+1);
            // check on whether to apply the penalty
            for(int j=0;j<=jMax;j++)
            {
                // if current value suggests apply penalty, adjust matrix equations
                if( vNew[j] < X - S[j] )
                {
                    bHat[j]=b[j]-penalty;dHat[j]=d[j] - penalty*(X - S[j]);
                }
                else
                {
                    bHat[j]=b[j];dHat[j]=d[j];
                }
            }
            // now solve *Hat matrix problem with thomas
            thomasSolve(a,bHat,c,dHat);
            // dHat now contains next guess at solution v_j^{q+1}
            // check for differences between dHat and vNew
            double error=0.;
            for(int j=0;j<=jMax;j++)
            {
                error += (dHat[j]-vNew[j])*(dHat[j]-vNew[j]);
            }
            // update vNew
            vNew = dHat;
            // make an exit condition when solution is converged
            if(error<tol*tol)
            {
                // cout << " # solved after "<< penaltyIt << " iterations\n\n" << endl;
                break;
            }
        }
        if(iter>=iterMax)
        {
            std::cout << " Error NOT converging within required iterations\n";
            std::cout.flush();
            throw;
        }
        // set old=new
        vOld=vNew;
    }
    int jStar=S0/dS;
    double sum=0.;
    sum+=(S0 - S[jStar])/dS * vNew[jStar+1];
    sum+=(S[jStar+1] - S0)/dS * vNew[jStar];
    return sum;
}

Now we declare our Black-Scholes parameters

In [4]:
// declare Black Scholes parameters
double S0,X,T,r,sigma;
// declare grid paramaters 
int iMax,jMax;
// declare local variables (ds,dt)
double SMax;

and initialise them

In [5]:
// initialise Black Scholes parameters
S0=9.735;X=10.;T=1.;r=0.05;sigma=0.4;
// initialise grid paramaters 
iMax=40;jMax=40;SMax=2*X;

before we go on check that our algorithm works and generates results.

In [6]:
cout << americanPut_CN(S0,X,T,r,sigma,iMax,jMax,SMax,1e6,1.e-6,1000);

1.47464

Next we can look at getting some results for different values of $n$ when $iMax=n$ and $jMax=n$.

In [7]:
MATH60082::tableRow("n","V_n");
MATH60082::emptyTableRow(2);
for(int k=1;k<=4;k++)
{
    int n=10*pow(2,k);
    iMax = n;
    jMax = n;
    double value = americanPut_CN(S0,X,T,r,sigma,iMax,jMax,SMax,1e6,1.e-6,1000);
    MATH60082::tableRow( n , value);
}

|             n|           V_n|
|--------------|--------------|
|            20|       1.47217|
|            40|       1.47464|
|            80|       1.47409|
|           160|       1.47467|


Obviously we do not have an analytical value for this option but there are many places that you can find an estimate for the value. Even on my website I have an online American put pricer written in javascript that estimates the value as 
$$
V_\text{exact}(S=9.735,t=0) = 1.47501 .
$$
so we can see that these estimates are not very far from the true value. If we are only interested in 4s.f. then just $n=160$ provides a good estimate. If we want something more accurate we will need to be more careful, including the interpolation and tolerance as before. 

So for the next example (with extrapolation) we use $S_0=X$, so that interpolation errors are removed and also increase $S_\text{max}$ and the tolerance. The new exact value we are aiming for is
$$
V_\text{exact}(S=X=10,t=0) = 1.36676 .
$$

In [8]:
double valueOld=0;

In [9]:
S0 = X;
MATH60082::tableRow("n","V_n","diff","V_extrap");
MATH60082::emptyTableRow(4);
for(int k=1;k<=5;k++)
{
    int n=10*pow(2,k);
    iMax = n;
    jMax = n;
    double value = americanPut_CN(S0,X,T,r,sigma,iMax,jMax,5*X,1e8,1.e-8,1000);
    double value_extrap = (4.*value - valueOld)/3.;
    MATH60082::tableRow(n,value,value - valueOld,value_extrap);
    valueOld = value;
}

|             n|           V_n|          diff|      V_extrap|
|--------------|--------------|--------------|--------------|
|            20|       1.27856|       1.27856|       1.70474|
|            40|       1.34674|     0.0681884|       1.36947|
|            80|        1.3615|     0.0147517|       1.36641|
|           160|       1.36539|    0.00389047|       1.36668|
|           320|       1.36641|    0.00102095|       1.36675|


Again we are able to get _quite_ smooth convergence although not quite as good as previously. This time the ratio of differences between the results is not perfect so the extrapolation doesn't perform quite as well. We are still able to improve our results though and the combination of $n=160$ and $n=320$ is able to give 5 almost 6s.f. of accuracy, whereas the standard method with $n=320$ gives only 3 or 4s.f. of accuracy.